In [1]:
import pandas as pd

# Load the datasets
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

# Merge the Transactions and Products datasets on ProductID
merged_transactions = pd.merge(transactions_df, products_df, on='ProductID', how='left')

# Merge the result with Customers dataset on CustomerID
merged_data = pd.merge(merged_transactions, customers_df, on='CustomerID', how='left')

# Check the first few rows of merged_data to ensure it's correctly merged
print(merged_data.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067   2024-04-25 7:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x                      ProductName     Category  Price_y  \
0      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
1      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
2      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
3      601.36   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
4      902.04   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   

      CustomerName         Region  SignupDate  
0   Andrea Jenkins         Europe  202

In [5]:
# Merge the Transactions and Products datasets, with custom suffixes to avoid name conflicts
merged_transactions = pd.merge(transactions_df, products_df[['ProductID', 'Price']], on='ProductID', how='left', suffixes=('_transaction', '_product'))

# Merge the result with the Customers dataset on CustomerID
merged_data = pd.merge(merged_transactions, customers_df, on='CustomerID', how='left')

# Check the columns to ensure they are correctly renamed
print(merged_data.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_transaction', 'Price_product',
       'CustomerName', 'Region', 'SignupDate'],
      dtype='object')


In [6]:
# Aggregate data by CustomerID: total spent, products bought, etc.
customer_features = merged_data.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    product_count=('ProductID', 'nunique'),
    avg_price=('Price_product', 'mean'),  # Use 'Price_product' instead of 'Price'
    region=('Region', 'first')
).reset_index()

# Convert SignupDate to a datetime object and calculate the signup duration
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
customers_df['SignupDuration'] = (pd.to_datetime('today') - customers_df['SignupDate']).dt.days

# Merge the features with the demographics
customer_features = pd.merge(customer_features, customers_df[['CustomerID', 'SignupDuration']], on='CustomerID')

# Check the final customer features dataframe
print(customer_features.head())


  CustomerID  total_spent  product_count   avg_price         region  \
0      C0001      3354.52              5  278.334000  South America   
1      C0002      1862.74              4  208.920000           Asia   
2      C0003      2725.38              4  195.707500  South America   
3      C0004      5354.88              8  240.636250  South America   
4      C0005      2034.24              3  291.603333           Asia   

   SignupDuration  
0             935  
1            1082  
2             329  
3             844  
4             899  


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Select the columns to use for similarity calculation (exclude CustomerID)
features = ['total_spent', 'product_count', 'avg_price', 'SignupDuration']
X = customer_features[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Compute the cosine similarity between customers
cosine_sim = cosine_similarity(X_scaled)

# Convert the cosine similarity matrix to a DataFrame for easier handling
cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

# Check the similarity scores between the first customer and others
print(cosine_sim_df['C0001'].sort_values(ascending=False).head(10))


CustomerID
C0001    1.000000
C0152    0.991213
C0070    0.948592
C0118    0.936542
C0011    0.914470
C0160    0.901967
C0103    0.883715
C0086    0.882640
C0166    0.880743
C0135    0.869706
Name: C0001, dtype: float64


In [8]:
# Function to get the top 3 similar customers for a given customer
def get_top_similar_customers(customer_id, cosine_sim_df, top_n=3):
    similar_customers = cosine_sim_df[customer_id].sort_values(ascending=False)[1:top_n+1]
    return similar_customers.index.tolist(), similar_customers.values.tolist()

# Create the Lookalike Model for the first 20 customers
lookalike_data = []

for customer_id in customer_features['CustomerID'][:20]:
    similar_customers, scores = get_top_similar_customers(customer_id, cosine_sim_df)
    for i in range(len(similar_customers)):
        lookalike_data.append({
            'CustomerID': customer_id,
            'LookalikeCustomerID': similar_customers[i],
            'SimilarityScore': scores[i]
        })

# Convert to DataFrame for better representation
lookalike_df = pd.DataFrame(lookalike_data)

# Create the Lookalike.csv file
lookalike_df.to_csv('Lookalike.csv', index=False)

# Display the first few rows of the lookalike recommendations
print(lookalike_df.head())


  CustomerID LookalikeCustomerID  SimilarityScore
0      C0001               C0152         0.991213
1      C0001               C0070         0.948592
2      C0001               C0118         0.936542
3      C0002               C0029         0.996958
4      C0002               C0027         0.980796
